# <font color='#B31B1'> Decision Trees </font>

One of the most popular algorithms for classification and regression are decision trees. The underlying idea is simple - we're going to build a set of rules in tree-form to classify a data point. For example, a simple decision tree to predict if you're falling asleep in class could be IF coffee_consumption == 0 THEN asleep_in_class = TRUE ELSE False. They're particularly popular because they're interpretable. (You can see a tree and know exactly how it maps from an input to a decision.)


* 2021-10-26 This notebook is authored by Madeleine Udell, available at https://github.com/ORIE4741/demos/blob/master/trees.ipynb
* 2023-04-08 Aleksandr Kazachkov modified for ESI 4611 at University of Florida

In [ ]:
from IPython.display import Image
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(rc={'figure.figsize':(6,6)}) 

## <font color='#B31B1'> Titanic Dataset </font>

We are going to use the [Titanic dataset](https://www.kaggle.com/c/titanic/data) for this example. We'll load it directly from the seaborn plotting package.

This dataset appears in one of the most basic and popular kaggle competitions: https://www.kaggle.com/c/titanic

In [ ]:
data = sns.load_dataset('titanic').drop(['alive'], axis = 1) #Removing some 'cheating' columns
data.head()

We process the dataset

In [ ]:
# We convert our categorical data into 1-Hot Columns 
# What this does is turns our cateogrical feature data into dummy variables.
# See, for example, this article - https://www.pluralsight.com/guides/handling-categorical-data-in-machine-learning-models
passengers = pd.get_dummies(data)

# Fill in NA values for age with the average
passengers.age = passengers.age.fillna(passengers.age.mean())

# Drop any rows with other missing values
# FILL IN HERE

passengers

In [ ]:
# Plot the distribution of deck, just for curiosity
sns.catplot(x='deck', data=data, kind='count', palette='summer')

In [ ]:
# We wish to predict the survival of a passenger
TARGET = "survived"
features = passengers.drop(columns=TARGET).columns

## <font color='#B31B1'> Decision Trees 'Manually' </font>
To get some intution for the decision tree algorithm, let's try to find an optimal root-node split for our dataset ourselves. For simplicity, consider a smaller version of our dataset with 2 features: age and gender.

In [ ]:
X = passengers[['age','sex_male','survived']]
X.head()

We first have to pick which measure of node purity we want to use - let's use Gini (but try implementing your own cross-entropy function for fun!).

In [ ]:
def gini(data, feature, split):
    
    left_node = data[data[feature] > split]
    p_left = left_node.survived.mean()
    N_left = left_node.shape[0]/data.shape[0]
    
    right_node = data[data[feature] <= split]
    p_right = right_node.survived.mean()

    
    return N_left*p_left*(1-p_left) + (1-N_left)*p_right*(1-p_right)

In [ ]:
best_impurity = np.Inf
best_split = None
best_split_feature = None

for feature in ['age','sex_male']: # search over features
    for s in X[feature].unique():  # search over thresholds
        impurity = # FILL IN HERE

print('Our best split is ', best_split_feature, '>', best_split)

We can investigate if our split led to a big change in node purity...

In [ ]:
passengers.survived.value_counts(normalize=True)

In [ ]:
passengers.query('sex_male > 0').survived.value_counts(normalize=True)

In [ ]:
passengers.query('sex_male <= 0').survived.value_counts(normalize=True)

In [ ]:
passengers.sex_male.value_counts()

Seems like gender is a good split for determining survival rate! We could repeat this process for two new nodes, one with all the male data, one for all the female and so on and so forth...

## <font color='#B31B1'> Decision Trees in Scikit-Learn </font>

Like most machine learning algorithms, the exceedingly smart people at scikit-learn have implemented decision trees in an optimized version of python called cython. In scikit-learn the tree based algorithms are in the `sklearn.tree` submodule.

Scikit-learn tree implementation [uses an optimized version of CART](http://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart) *(Classification and Regression Trees)*, that allows us to use the decision trees for both classification and regression.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

tree = DecisionTreeClassifier()

In [ ]:
# Set up X and Y, and do train/test split
X = # FILL IN HERE
Y = # FILL IN HERE

# Train/test split
X_train, Y_train, X_test, Y_test = # FILL IN HERE

tree.fit(X_train, Y_train)

In [ ]:
tree.predict(passengers[features])[:10]

Instead of doing one train-test split of our data, we can use **cross-validation** to get an 'average' performance over k splits of our dataset. Scikit-learn does that automaticaly for us using the `cross_val_score` function. Here 'cv' is the parameter to indicate the number of splits. We can even specify what error metric we want to look at (i.e. accuracy vs. AUC)

In [ ]:
cross_val_score(tree, X, Y,
                scoring="accuracy", 
                cv=5).mean()

One very good advantage that decision trees have is that they can be visualized, and we can explain why they take a certain decision  (we say trees have a high **explainability**). Scikit-learn trees can be visualized with `Graphviz`, a graph visualization tool. See https://www.graphviz.org/download/.

You can install it using `conda install -c anaconda graphviz` followed by `conda install python-graphviz`. On a Mac, if you have homebrew, use `brew install graphviz`.

If we have installed graphviz we can plot the tree directly.

In [ ]:
%pip install graphviz

In [ ]:
import graphviz
from sklearn.tree import export_graphviz

def draw_tree(tree):
    dot_data = export_graphviz(tree, out_file=None, 
                         feature_names=features, 
                               class_names=['survived', 'died'],
                         filled=True, 
                         #impurity=True,
                         rounded=True,  
                         special_characters=True,
                              proportion = True)  #trying changing proportion = False
    
    graph = graphviz.Source(dot_data)
    graph.format = 'png'
    graph.render('tree',view=True)

In [ ]:
draw_tree(tree)

In [ ]:
# We can always check out the help docs
help(DecisionTreeClassifier)

These are the most important hyperparameters for scikit-learn `DecisionTreeClassifier`:

Here is a great article that goes over how to understand and potentially use these for tuning your model ... 
https://towardsdatascience.com/how-to-tune-a-decision-tree-f03721801680


* **criterion** : The partition criterion to use, we can use either `gini`, or `entropy` 

* **max_depth** (int>1) : The max depth the tree can achieve. We define as depth as the number of nodes an observation goes through (how many *questions* are asked).

* **max_features** (int or float(percentage)):  The maximum number of potential partitions evaluated when we split a node.

* **max_leaf_nodes** (int or None): Max number of leaves in the tree.

* **min_impurity_decrease** (float) : The minimum information gain required in a node to split it (if no feature provides that minimum, the node becomes a leaf).

* **class_weight** : For imbalanced classes, we can use `class_weight`, which is a dictionary with the shape `{class: weight}`, so sklearn takes the class weights into consideration. We can also use `class_weight=balanced` so sklearn creates the weights automatically based on class distribution.

Decision trees are prone to overfitting, there are some hyperparameters that help us control this:

* **min_samples_leaf** (int or float(percentage)) : Minimum number of observations on a node to consider the node a leaf. Default value is 1, this means that by default sklearn will create leaves with one observation (and this memorize the dataset).

* **min_samples_split** (int or float(percentage)) : Minimum number of observations on a node to generate a partition. By default is 2, this means sklearn will split all nodes with 2 or more observations by default.

For example, we can create a simpler tree by setting the maximum depth

In [ ]:
simple_tree = # FILL IN HERE

In [ ]:
simple_tree.fit(# FILL IN HERE)

In [ ]:
draw_tree(simple_tree)

In [ ]:
cross_val_score(simple_tree, passengers.drop(TARGET, axis=1), 
                passengers.survived, scoring="roc_auc", cv=3).mean()

So we see this simple tree performs much better than the initial tree (that was overfitting), and it is also very simple to explain!

But how do we know what the optimal depth is?

Well this is a balance of practicality and "hyperparameter tuning" ... 

In [ ]:
# Perhaps for our business problem anything with greater than depth of 10 is too complicated and don't want to run the risk of overfitting (think of our first tree...)
# So let's test a range of depths from 2 to 10 using a for loop

depths = np.arange(2,10) # define the depths
results = [] # create an empty data frame for our results

for depth in depths:
    best_depth_tree = # FILL IN HERE # creating an instance of a decision tree
    results.append(cross_val_score(# FILL IN HERE, 
                    scoring="roc_auc", # getting the cv roc_auc metric for the tree at each depth
                cv=3).mean())



In [ ]:
test = pd.DataFrame({'depths':depths, 'mean_roc_auc':results})
test.sort_values("mean_roc_auc", ascending=False)
plt.plot(test.depths, test.mean_roc_auc)

So I would now move forward with my model with max_depth set = 3